In [12]:
import random
import w2v_utils  as w2v_utils
import numpy as np
import gensim.models
from gensim.models.word2vec import Word2Vec
import random
from __future__ import division
import math 
import re

def prepareData(path,trainExamples):
    
    trainData=[]
    fp=open(path, 'rb')
    data=fp.readline()
    data=fp.readline()
    while data!="":
        trainData.append(data.split('##'))
        data=fp.readline()
    data=[]
    labels=[] 
    lb=[]
    random.shuffle(trainData)
    for record in trainData:
        data.append(record[0])
        lb.append(record[1].split('\t'))
        labels.append(lb[0][0])
        lb=[]
    train_set, test_set, train_label, test_label = data[:trainExamples], data[trainExamples:],labels[:trainExamples],labels[trainExamples:]
    
        
    return [train_set, test_set, train_label, test_label]
    
    
train_set, test_set, train_labels, test_labels=prepareData('Case Advanced Find View (1).txt',7000)
print 'Size of Training Set: ',len(train_set)
print 'Size of Training Labels: ',len(train_labels)
print 'Size of Test Set: ',len(test_set)
print 'Size of Test Labels: ',len(test_labels)

trainingSentences = [w2v_utils.review_to_wordlist(review,True) for review in train_set]
testSentences     = [w2v_utils.review_to_wordlist(review,True) for review in test_set]


word2vecInput = trainingSentences+testSentences


from gensim.models import word2vec

'''
num_features = 300    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


print "Training model..."
model = gensim.models.Word2Vec(workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

model.build_vocab(word2vecInput)#build vocablary from all training words
model.train(word2vecInput)
model.save('word2vec')
print "Finish training model..."
'''
model = Word2Vec.load('word2vec')
# preparing  data
x_train,Y_train,x_val,Y_val =  w2v_utils.splitDataset(train_set,train_labels, split=0.999) 
x_test  = test_set
Y_test  = test_labels
x_train = w2v_utils.clean(x_train, True)
x_val = w2v_utils.clean(x_val,True)
x_test = w2v_utils.clean(x_test,True)

Y_train = [re.sub('["]', '', label) for label in Y_train]
Y_val   = [re.sub('["]', '', label) for label in Y_val]
Y_test  = [re.sub('["]', '', label) for label in Y_test]

Size of Training Set:  7000
Size of Training Labels:  7000
Size of Test Set:  2977
Size of Test Labels:  2977


In [13]:
######## getting sentence vectors
print "Process of word to vector conversion initiated!"
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVector(text, size, debug = False):        
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            wordVector = model[word].reshape((1, size))
            vec += wordVector
            count += 1.
        except KeyError:
            if debug:
                print "not found! ",  word
                count += 1.
            continue
    if count != 0:     
        #Sentence vectors
        vec_avg    = vec/count   
    return vec

######## Converting training, validation and test sentences into vectors
x_train = np.concatenate([buildWordVector(sentence, model.layer1_size, debug = False) for sentence in x_train])
print "Training set Sentences word vector dimensions: \n",x_train.shape

x_val = np.concatenate([buildWordVector(sentence, model.layer1_size, debug = False) for sentence in x_val]) 
print "Validation set Sentences word vector dimensions: \n",x_val.shape
    
x_test = np.concatenate([buildWordVector(sentence, model.layer1_size, debug = False) for sentence in x_test])
print "Gold Sentences word vector dimensions: \n",x_test.shape



############################### Preparing Data with One Hot #######################################
X_train = x_train #training Data
y_train = w2v_utils.oneHotVectors(Y_train)
#y_train = w2v_utils.categoriesToLabels(Y_train)
y_train = np.nonzero(y_train)[1]
print "Training Data: ",X_train.shape, y_train.shape

###############################Preparing Validation Data
X_val = x_val
y_val = w2v_utils.oneHotVectors(Y_val)
#y_val = w2v_utils.categoriesToLabels(Y_val)
y_val   = np.nonzero(y_val)[1]
print "Validation Data: ",X_val.shape, y_val.shape

###############################Preparing Test Gold Data
X_test = x_test
y_test = w2v_utils.oneHotVectors(Y_test)
#y_test = w2v_utils.categoriesToLabels(Y_test)
y_test  = np.nonzero(y_test)[1]

print "Gold Test Data: ", X_test.shape,y_test.shape

Process of word to vector conversion initiated!
Training set Sentences word vector dimensions: 
(6993, 300)
Validation set Sentences word vector dimensions: 
(7, 300)
Gold Sentences word vector dimensions: 
(2977, 300)
Training Data:  (6993, 300) (6993,)
Validation Data:  (7, 300) (7,)
Gold Test Data:  (2977, 300) (2977,)


In [14]:
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

#correctPred = (y_test != y_pred).sum()
#print "Total test instances: ",len(y_test),"Total Correctly Predict classes: ",correctPred

print "--------------------------------------------------------------"
print "Confusion Matrix: \n\n",
print w2v_utils.confusionMatrix(y_test, y_pred)

print "--------------------------------------------------------------"
print "Classification  Report: \n\n",
target_names = ['Research', 'Bug']
print(classification_report( y_test,y_pred, target_names=target_names))

--------------------------------------------------------------
Confusion Matrix: 

Predicted     0    1   All
Actual                    
0          1567  652  2219
1           440  318   758
All        2007  970  2977
--------------------------------------------------------------
Classification  Report: 

             precision    recall  f1-score   support

   Research       0.71      0.78      0.74      2007
        Bug       0.42      0.33      0.37       970

avg / total       0.61      0.63      0.62      2977



In [15]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn import svm

In [16]:
clf = svm.SVC(kernel='linear', C = 1.0)

In [17]:
clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
y_pred = clf.predict(X_test)
print "--------------------------------------------------------------"
print "Confusion Matrix: \n\n",
print w2v_utils.confusionMatrix(y_test, y_pred)

print "--------------------------------------------------------------"
print "Classification  Report: \n\n",
target_names = ['Research', 'Bug']
print(classification_report( y_test,y_pred, target_names=target_names))

--------------------------------------------------------------
Confusion Matrix: 

Predicted     0    1   All
Actual                    
0          1948  868  2816
1            59  102   161
All        2007  970  2977
--------------------------------------------------------------
Classification  Report: 

             precision    recall  f1-score   support

   Research       0.69      0.97      0.81      2007
        Bug       0.63      0.11      0.18       970

avg / total       0.67      0.69      0.60      2977

